# Compute arbitrary terms - high-level generic assembly procedures - Generic Weak-Form Language (GWFL)

In [1]:
import getfem as gf
import numpy as np

elements_degree = 1
# elements_degree = 2

mesh = gf.Mesh("cartesian", np.arange(0.0, 2.0, 1.0))

`mf` is supposed to be an already declared `gf.MeshFem` object and `mim` a already declared `gf.MeshIm` object on the same mesh.

In [2]:
mf = gf.MeshFem(mesh, 1)
mf.set_classical_fem(elements_degree)
print(mf)


BEGIN MESH_FEM

QDIM 1
 CONVEX 0 'FEM_PK(1,1)'
 BEGIN DOF_ENUMERATION 
  0:  0 1
 END DOF_ENUMERATION 
END MESH_FEM



In [3]:
mim = gf.MeshIm(mesh, elements_degree*2)
print(mim)


BEGIN MESH_IM

 CONVEX 0 'IM_GAUSS1D(2)'
END MESH_IM



As a first example, if one needs to perform the assembly of a Poisson problem

$$
-\mathrm{div} \nabla u=f,\mathrm{in} \Omega ,
$$

the stiffness matrix is given

$$
K_{i,j} =\int _{\Omega } \nabla \varphi _{i} \cdot \nabla \varphi _{j} dx,
$$

and will be assembled by the following code:

In [4]:
md = gf.Model("real")
md.add_fem_variable("u", mf)
md.add_nonlinear_term(mim, "Grad_u.Grad_Test_u")
md.assembly(option="build_matrix")
K = md.tangent_matrix()
print(K)

matrix(2, 2)
( (r0, 1) (r1, -1) )
( (r0, -1) (r1, 1) )



Note that the value of the variable do not really intervene because of the linearity of the problem.
This allows to pass `"u"` as the value of the variable which will not be used.
Note also that two other possible expressions for exactly the same result for the assembly string are `"Grad_Test2_u.Grad_Test_u"` (i.e. an order 2 expression) or `"Norm_sqr(Grad_u)/2"` (i.e. a potential). 
In fact other possible assembly string will give the same result such as `"Grad_u.Grad_u/2"` or `"[Grad_u(1), Grad_u(2)].[Grad_Test_u(1), Grad_Test_u(2)]"` for two-dimensional problems.
However, the recommendation is preferably to give an order 1 expression (weak formulation) if there is no particular reason to prefer an order 0 or an order 2 expression.

In [5]:
md = gf.Model("real")
md.add_fem_variable("u", mf)
md.add_nonlinear_term(mim, "Grad_u.Grad_u/2")
md.assembly(option="build_matrix")
K = md.tangent_matrix()
print(K)

matrix(2, 2)
( (r0, 1) (r1, -1) )
( (r0, -1) (r1, 1) )



In [6]:
# for two-dimensional problems
# md = gf.Model("real")
# md.add_fem_variable("u", mf)
# md.add_nonlinear_term(mim, "[Grad_u(1), Grad_u(2)].[Grad_Test_u(1), Grad_Test_u(2)]")
# md.assembly(option="build_matrix")
# K = md.tangent_matrix()
# print(K)

As a second example, let us consider a coupled problem, for instance the mixed problem of incompressible elasticity given by the equations

$$
-\mathrm{div}\left( \mu \left( \nabla u+( \nabla u)^{T}\right) -pI_{d}\right) =f,\mathrm{in} \thinspace \Omega , \\
-\mathrm{div} \thinspace u=0.
$$

where `u` is the vector valued displacement and `p` the pressure. The assembly of the matrix for the whole coupled system can be performed as follows: